In [1]:
# for reference see this code: /home/csutter/DRIVE/weather_img_concatmodels/cnn_hrrr_fcsthr2/nested_skill_streamline_vs_blending.ipynb

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)        # Use full width of the console
pd.set_option("display.max_colwidth", None)  # Show full content in each column

# CNN model selection

This file: /home/csutter/DRIVE-clean/CNN/data_results/results_by_exp_innerVal.csv 
- Has Baseline model performance appended for each of the 36 BL models (hyperparams L2 and dropout set to 0)
- Has HT model performance appended for each of the 64 HT models (16 hyperparam sets x top 4 BL models)
- For final model selection, consider all 36 (with initial HTs of 0s) + 64 (with other HTs tuned) for final selection - this is why all models were appended into the same csv file.

### CNN - Baseline selection (BL)

In [2]:
##### For Baseline top 4 model selections

df = pd.read_csv("/home/csutter/DRIVE-clean/CNN/data_results/results_by_exp_innerVal.csv")

# Important! If recreating the selection from the 36 baseline models, run this code below to subset just the first 36 runs. O/w if doing final selection post hyptuning, don't need to subset - want to consider all models. See markdown note above for more detail.
df = df.head(36)

df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)


top10_avgrecall = df.sort_values("avg_recall", ascending=False)[0:10]
selected4 = top10_avgrecall.sort_values("overall", ascending=False)[0:4]

display(selected4)

print(selected4["exp_details"])

,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
20,107445,74272,7635,5651,14945,8968,42290,31316,38000,25133,4575,3204,97697,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue,2025-07-01 13:22:20.463990,0.691256,0.661395,0.700328,0.600067,0.740144,0.740506,0.688488
2,107445,73410,7635,5625,14945,8582,42290,30997,38000,25063,4575,3143,97679,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:06:33.552986,0.683233,0.659553,0.686995,0.574239,0.736739,0.732963,0.678098
26,107445,73123,7635,5655,14945,8844,42290,30146,38000,25309,4575,3169,97397,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue,2025-07-01 14:03:48.795248,0.680562,0.666026,0.692678,0.591770,0.740668,0.712840,0.680796
4,107445,72893,7635,5761,14945,9210,42290,30253,38000,24391,4575,3278,96548,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse,2025-06-29 03:20:17.279312,0.678422,0.641868,0.716503,0.616260,0.754551,0.715370,0.688910


20        _A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugTrue
2        _A_resnet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse
26         _A_resnet_TRLETrue_ASTTrue_L20_DR0_E75_AugTrue
4     _A_mobilenet_TRLETrue_ASTFalse_L20_DR0_E75_AugFalse
Name: exp_details, dtype: object


### CNN - Hyperparameter tuning (HT) selection

In [4]:
##### for HT model selection

# for each of the 4 BL models that hyp tuning was performed, grab the best model from each, and move forward with HTing EACH of those for the + weather data. 

df = pd.read_csv("/home/csutter/DRIVE-clean/CNN/data_results/results_by_exp_innerVal.csv")

df["overall"] = df["correctims"]/df["totalims"]

for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)


# add col with the BL model type (i.e., the 4 BL models) from which we are then selecting the best one hyperparams
model_BL_desc = []
for i in df["exp_details"]:
    b = i.find("L2")
    e = i.rfind("Aug")
    desc = i[:b]+i[e:]
    model_BL_desc.append(desc)

df["model_BL_desc"] = model_BL_desc

unique_BL = np.unique(df["model_BL_desc"])


### The way I originally did it with selecting top one model PER each baseline model
for bl in unique_BL:
    dfsub = df[df["model_BL_desc"]==bl]
    if len(dfsub)>1: # only look at the selected 4 BL models, not all the other 32 with only the one BL run exp
        print(len(dfsub))
        top4_avgrecall = dfsub.sort_values("avg_recall", ascending=False)[0:5] # since only 16 HT are run, select from top 5 (roughly 1/3)
        selected1 = top4_avgrecall.sort_values("overall", ascending=False)[0:1] # grab the top 1
        print(bl)
        display(selected1)
        print(selected1["exp_details"])

17
_A_mobilenet_TRLETrue_ASTFalse_AugFalse


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc
57,107445,71893,7635,5911,14945,9531,42290,29259,38000,23703,4575,3489,95897,nestcv_5cat_twotrain,_A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse,2025-07-08 18:38:11.556300,0.669114,0.623763,0.762623,0.637738,0.774198,0.691866,0.698038,_A_mobilenet_TRLETrue_ASTFalse_AugFalse


57    _A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse
Name: exp_details, dtype: object
17
_A_resnet_TRLETrue_ASTFalse_AugFalse


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc
87,107445,72769,7635,5874,14945,9112,42290,29897,38000,24177,4575,3709,96736,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,2025-07-14 02:57:04.840090,0.677267,0.636237,0.81071,0.609702,0.769352,0.706952,0.706591,_A_resnet_TRLETrue_ASTFalse_AugFalse


87    _A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse
Name: exp_details, dtype: object
17
_A_resnet_TRLETrue_ASTFalse_AugTrue


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc
74,107445,73460,7635,5787,14945,9243,42290,29897,38000,25025,4575,3508,97365,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue,2025-07-14 01:21:32.194943,0.683699,0.658553,0.766776,0.618468,0.757957,0.706952,0.701741,_A_resnet_TRLETrue_ASTFalse_AugTrue


74    _A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue
Name: exp_details, dtype: object
17
_A_resnet_TRLETrue_ASTTrue_AugTrue


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,rundatetime,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,model_BL_desc
39,107445,74274,7635,5909,14945,9010,42290,30413,38000,25346,4575,3596,97484,nestcv_5cat_twotrain,_A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,2025-07-08 11:27:22.701127,0.691275,0.667,0.786011,0.602877,0.773936,0.719153,0.709795,_A_resnet_TRLETrue_ASTTrue_AugTrue


39    _A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue
Name: exp_details, dtype: object


# Downstream model
- After models are run (see dir: /home/csutter/DRIVE-clean/downstream), results are appended to here: /home/csutter/DRIVE-clean/downstream/data_results/results_by_each_run_innerVal.csv
- First chunk below aggregates results by model since the runs were appended by fold, which we need to aggregate across.
- Then can select, similar to baseline/HT selection of CNN, to select best from that aggregated file -- last chunk of code

In [2]:
# Already ran this aggregation code, don't need to run again!

# Important first step: That csv has each model|fold innerVal results appended, but need to aggregate across the 30 folds to make selection of best model, so FIRST need to preprocess to aggregate at the model level, rather than model|fold level

results_model_fold = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_each_run_innerVal.csv")

results_model_fold.head(4)

aggregated = results_model_fold.groupby(["exp_desc","exp_details"]).sum().reset_index()
aggregated.head(5)

# Save this out so don't need to run again
aggregated.to_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

/tmp/tmp.80hJAYorXg/ipykernel_4194109/4133674865.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated = results_model_fold.groupby(["exp_desc","exp_details"]).sum().reset_index()


In [3]:
results_model_fold = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_each_run_innerVal.csv")

results_model_fold.head(4)

,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,exp_desc,exp_details,datafold
0,3697,2814,294,178,511,297,1422,1154,1326,1086,144,99,3451,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"logistic_{'max_iter': 100, 'C': 0.01}",nestcv_5cat_twotrain_OT1_m3_T5V0
1,3697,2800,294,177,511,304,1422,1140,1326,1086,144,93,3448,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"logistic_{'max_iter': 100, 'C': 0.1}",nestcv_5cat_twotrain_OT1_m3_T5V0
2,3697,2789,294,175,511,308,1422,1122,1326,1100,144,84,3431,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"logistic_{'max_iter': 100, 'C': 1.0}",nestcv_5cat_twotrain_OT1_m3_T5V0
3,3697,2791,294,177,511,307,1422,1123,1326,1101,144,83,3429,A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse,"logistic_{'max_iter': 100, 'C': 10.0}",nestcv_5cat_twotrain_OT1_m3_T5V0


In [4]:
print(np.unique(results_model_fold["exp_desc"]))

['A_mobilenet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugFalse'
 'A_mobilenet_TRLETrue_ASTFalse_L21e-05_DR0_4_E75_AugFalse'
 'A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue'
 'A_resnet_TRLETrue_ASTFalse_L20_1_DR0_2_E75_AugFalse'
 'A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue'
 'A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue']


### Downstream model selection

In [5]:
df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]



df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

# grab one from  a specific arch
archs = []
for i in range(0,len(df)):
    desc = df["exp_desc"][i]
    desc = desc[2:]
    end = desc.find("_")
    a = desc[:end]
    archs.append(a)

df["arch"] = archs

print(len(df))


# Decision: index 1507 showed up in a variety of different weighted-importances (when we tried these below), and replaced the initial top 10 with weighted recall rather than just plain average. Motivation to keep it simple and just move forward with the simplest -- top 10 avg recall, then take the max overall from that. Top models are all pretty similar, anyway. 
top10_avgrecall = df.sort_values("avg_recall", ascending=False)[0:10]
selected4 = top10_avgrecall.sort_values("overall", ascending=False)[0:1]

display(selected4)

print(selected4["exp_details"])
print(np.max(selected4["perc_snow"]))

# Look at that from each arch.. maybe justify taking the mobilenet one??
# Decision: index 1507 showed up in a variety of different weighted-importances (when we tried these below), and replaced the initial top 10 with weighted recall rather than just plain average. Motivation to keep it simple and just move forward with the simplest -- top 10 avg recall, then take the max overall from that. Top models are all pretty similar, anyway. 
resttop = df.sort_values("avg_recall", ascending=False)[0:10]
selected4 = top10_avgrecall.sort_values("overall", ascending=False)[0:1]

display(selected4)

print(selected4["exp_details"])
print(np.max(selected4["perc_snow"]))

2742


,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch
2421,2421,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86690,7635,6114,14945,11190,42290,32202,38000,33864,4575,3320,102107,0.806831,0.891158,0.725683,0.748745,0.800786,0.761457,0.785566,resnet


2421    rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}
Name: exp_details, dtype: object
0.748745399799264


,Unnamed: 0,exp_desc,exp_details,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,nims_poor_viz,correct_poor_viz,ok,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall,arch
2421,2421,A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue,"rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}",107445,86690,7635,6114,14945,11190,42290,32202,38000,33864,4575,3320,102107,0.806831,0.891158,0.725683,0.748745,0.800786,0.761457,0.785566,resnet


2421    rf_{'max_depth': 10, 'max_samples': 0.5, 'n_estimators': 300, 'max_features': 3, 'min_samples_leaf': 5, 'bootstrap': True}
Name: exp_details, dtype: object
0.748745399799264


In [6]:
len(df)

2742